In [1]:
# importy
import shutil
import os
import pandas as pd

import sqlalchemy as sql
from sqlalchemy import create_engine, select, MetaData, Table
from sqlalchemy import Column, Text, Integer, Float, ForeignKey, PrimaryKeyConstraint
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

In [2]:
# Zmienne globalne
dbTemplateName = "InitialDatabase.db"
dbName = "StockData.db"

In [3]:
# Usuwanie starej instancji bazy danych jeżeli istnieje
if os.path.isfile(dbName):
    print("Usuwanie starej bazy: " + dbName)
    os.remove(dbName)
# Kopiowanie pliku szablonu bazy danych
print("Kopiowanie szablonu bazy danych")
shutil.copyfile(os.path.join("csv", dbTemplateName), dbName)

Usuwanie starej bazy: StockData.db


PermissionError: [WinError 32] Proces nie może uzyskać dostępu do pliku, ponieważ jest on używany przez inny proces: 'StockData.db'

In [4]:
# Nawiązywanie połączenia z bazą
db_string = 'sqlite:///' + dbName
engine = create_engine(db_string)
metadata = MetaData()
insp = sql.inspect(engine)

session = (sessionmaker(bind=engine))()
Base = declarative_base()

# Pobieranie listy tablic
print("Lista tablic:", insp.get_table_names())

Lista tablic: ['Companies', 'Records']


In [5]:
# Tworzenie klas danych
class Companies(Base):
    __tablename__ = 'Companies'
    CompanyID = Column(Integer, primary_key=True)
    CompanyName = Column(Text)
    CompanyType = Column(Text)
    CompanyDate = Column(Integer)
    
    def __str__(self):
        CompanyInfo = self.CompanyName + " " + self.CompanyType + ", zołożono: "
        print("GeneratedInfo", CompanyInfo)
        return CompanyInfo


class Records(Base):
    __tablename__ = 'Records'
    RecordID = Column(Integer, primary_key=True)
    CompanyID = Column(Integer, ForeignKey('Companies.CompanyID'))
    Date  = Column(Integer)
    Open  = Column(Float)
    High  = Column(Float)
    Low   = Column(Float)
    Close = Column(Float)
    Volume = Column(Integer)

In [6]:
# Pozyskiwanie listy plików csv
CsvList = os.listdir('csv')
CsvList.remove(dbTemplateName)
print("Lista plików csv z danymi: ", CsvList)

# Ładowanie plików
CsvData = {}
for i in CsvList:
    CsvData[i[:-4]] = pd.read_csv(os.path.join('csv', i), header=0)
print("Załadowano %i plików csv z danymi typu %s" % (len(CsvList), str(type(CsvData[CsvList[0][:-4]]))))

Lista plików csv z danymi:  ['Enea.csv', 'Energa.csv', 'PGE.csv', 'Tauron.csv', 'WIG20.csv']
Załadowano 5 plików csv z danymi typu <class 'pandas.core.frame.DataFrame'>


In [7]:
# Zapisywanie danych do bazy

# Tworzenie klucza głównego
key = 1

# Interacja po danych
for company in CsvData.keys():
    # Pozyskiwanie ID firmy
    id = session.query(Companies).filter_by(CompanyName = company).all()[0].CompanyID
    # Pozyskiwanie listy elementów
    RowCount = CsvData[company].index

    # Pozyskiwanie łącz do kolumn danych
    Date_col = CsvData[company]["Date"]
    Open_col  = CsvData[company]["Open"]
    High_col  = CsvData[company]["High"]
    Low_col   = CsvData[company]["Low"]
    Close_col = CsvData[company]["Close"]
    Volume_col = CsvData[company]["Volume"]

    # Tworzenie listy dodawanych elementów
    RowsToInsert = []
    for i in RowCount:
        RowsToInsert.append(Records(RecordID = key,
            CompanyID = id,
            Date  = int(Date_col[i][:-9].replace('-', '')),
            Open  = float(Open_col[i].replace(',', '.')),
            High  = float(High_col[i].replace(',', '.')),
            Low   = float(Low_col[i].replace(',', '.')),
            Close = float(Close_col[i].replace(',', '.')),
            Volume = int(Volume_col[i])))
        key += 1
    # Wgrywanie i zapis danych
    session.add_all(RowsToInsert)
    session.commit()
        


In [ ]:
# Testowe pobranie danych
# Dotyczy wszystkich bloków poniżej

In [5]:
session_task = session.query(Records).filter_by(CompanyID = 1).filter(Records.Date < 20220104)
task_result = session_task.all()
print(task_result)

In [11]:
mapper_task = select(Companies.CompanyName)
print('Mapper join: ')
print(mapper_task)
mapper_results = engine.execute(mapper_task).fetchall()
print('--------------------------------------------------')
companiesList = [mapper_results[i][0] for i in range(len(mapper_results))]
print(["brak"] + companiesList)
print(tuple(["brak"] + companiesList))


Mapper join: 
SELECT "Companies"."CompanyName" 
FROM "Companies"
--------------------------------------------------
['brak', 'WIG20', 'Tauron', 'Energa', 'PGE', 'Enea']
('brak', 'WIG20', 'Tauron', 'Energa', 'PGE', 'Enea')


In [17]:
companydata = session.query(Records)\
        .filter_by(CompanyID = 1)\
        .filter(Records.Date > 20220101)\
        .filter(Records.Date < 20220301)\
        .all()

datX = [companydata[i].Date\
    for i in range(len(companydata))]
print(datX)

[20220103, 20220104, 20220105, 20220107, 20220110, 20220111, 20220112, 20220113, 20220114, 20220117, 20220118, 20220119, 20220120, 20220121, 20220124, 20220125, 20220126, 20220127, 20220128, 20220131, 20220201, 20220202, 20220203, 20220204, 20220207, 20220208, 20220209, 20220210, 20220211, 20220214, 20220215, 20220216, 20220217, 20220218, 20220221, 20220222, 20220223, 20220224, 20220225, 20220228]


In [7]:
print(session.query(Companies).filter_by(CompanyName = "Enea").all()[0].__str__())

GeneratedInfo Enea s.a., zołożono: 
Enea s.a., zołożono: 


In [8]:
os.system("pyinstaller --help")

0